In [1]:
import os
import numpy as np
from scipy.misc import imread, imresize
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from random import shuffle
import glob
import re

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn import preprocessing

%matplotlib inline

In [2]:
DATA_DIR = 'D:/datasets/caltech/101_ObjectCategories'
INPUT_SIZE = 224

In [3]:
VALID_IMAGE_FORMATS = frozenset(['jpg', 'jpeg'])
def create_image_lists(image_dir, train_percent):
    if not os.path.isdir(image_dir):
        raise ValueError("Image directory {} not found.".format(image_dir))
    image_lists = {}
    sub_dirs = [x[0] for x in os.walk(image_dir)]
    sub_dirs_without_root = sub_dirs[1:]  # first element is root directory
    num_classes=0
    for sub_dir in sub_dirs_without_root:
        file_list = []
        dir_name = os.path.basename(sub_dir)
        if dir_name == image_dir:
            continue
        #print("Looking for images in '{}'".format(dir_name))
        for extension in VALID_IMAGE_FORMATS:
            file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
            file_list.extend(glob.glob(file_glob))
        if not file_list:
            continue
        num_classes+=1
        label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
        training_images = []
        validation_images = []
        shuffle(file_list)
        if train_percent<1:
            train_cnt=int(math.ceil(train_percent*len(file_list)))
            #print(label_name,train_percent,len(file_list),train_cnt)
        else:
            train_cnt=train_percent
            
        for i,file_name in enumerate(file_list):
            base_name = os.path.basename(file_name)
            if i < train_cnt:
                training_images.append(base_name)
            #elif i<train_cnt+15:
            else:
                validation_images.append(base_name)
        image_lists[label_name] = {
            'dir': dir_name,
            'training': training_images,
            'validation': validation_images,
        }
    return image_lists,num_classes

In [4]:
image_lists,num_classes = create_image_lists(DATA_DIR, train_percent=30)
print(num_classes)

101


In [5]:
from keras.models import Model,Sequential, load_model,model_from_json
from keras.applications import mobilenet,mobilenet_v2,densenet,inception_resnet_v2,inception_v3,resnet_v2
from keras.utils.generic_utils import CustomObjectScope
from keras.layers import Flatten, Dense, Dropout,GlobalAveragePooling2D,Activation, Conv2D, Reshape,DepthwiseConv2D,Input
from keras.optimizers import SGD, Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, CSVLogger, EarlyStopping
from keras.metrics import top_k_categorical_accuracy
import numpy as np
from sklearn.metrics import confusion_matrix
import keras.applications
from keras.preprocessing.image import (ImageDataGenerator, Iterator,
                                       array_to_img, img_to_array, load_img)
from keras import backend as K

#from myimage import ImageDataGenerator

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

Using TensorFlow backend.


In [ ]:
input_shape=(INPUT_SIZE,INPUT_SIZE,3)
if True:
    net_model=inception_v3
    net_description='inception_v3'
    base_model = inception_v3.InceptionV3(input_shape=input_shape, include_top=True, weights='imagenet')
else:
    net_model=mobilenet
    net_description='mobilenet'
    base_model = mobilenet.MobileNet(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
    #base_model = mobilenet_v2.MobileNetV2(alpha=1.4, input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
    #base_model = densenet.DenseNet121(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')
    #base_model = inception_resnet_v2.InceptionResNetV2(input_shape=input_shape, include_top=True, weights='imagenet', pooling='avg')            
preprocessing_function=net_model.preprocess_input
x=base_model.layers[-2].output
#base_model.summary()

#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax', use_bias=True,name='preds')(x)
model=Model(base_model.inputs, x)

In [15]:
import efficientnet.keras as enet
base_model = enet.EfficientNetB5(weights=None)
base_model.load_weights('enet_pretrained/efficientnet-b5-weights.h5') #-train
x=base_model.layers[-2].output
#base_model.summary()

#x = Dense(1024, activation='relu')(x)
#x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax', use_bias=True,name='preds')(x)
model=Model(base_model.inputs, x)

INPUT_SIZE = model.input_shape[1]
input_shape=(INPUT_SIZE,INPUT_SIZE,3)

preprocessing_function=enet.preprocess_input
net_description='enet5_train'

model.summary()
print(INPUT_SIZE)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 456, 456, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 228, 228, 48) 1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 228, 228, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 228, 228, 48) 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

In [8]:
class CustomImageDataGenerator(ImageDataGenerator):
    def flow_from_image_lists(self, image_lists,
                              category, image_dir,
                              target_size=(256, 256), color_mode='rgb',
                              class_mode='categorical',
                              batch_size=32, shuffle=True, seed=None,
                              save_to_dir=None,
                              save_prefix='',
                              save_format='jpeg'):
        return ImageListIterator(
            image_lists, self,
            category, image_dir,
            target_size=target_size, color_mode=color_mode,
            class_mode=class_mode,
            data_format=self.data_format,
            batch_size=batch_size, shuffle=shuffle, seed=seed,
            save_to_dir=save_to_dir,
            save_prefix=save_prefix,
            save_format=save_format)


class ImageListIterator(Iterator):
    """Iterator capable of reading images from a directory on disk.

    # Arguments
        image_lists: Dictionary of training images for each label.
        image_data_generator: Instance of `ImageDataGenerator`
            to use for random transformations and normalization.
        target_size: tuple of integers, dimensions to resize input images to.
        color_mode: One of `"rgb"`, `"grayscale"`. Color mode to read images.
        classes: Optional list of strings, names of sudirectories
            containing images from each class (e.g. `["dogs", "cats"]`).
            It will be computed automatically if not set.
        class_mode: Mode for yielding the targets:
            `"binary"`: binary targets (if there are only two classes),
            `"categorical"`: categorical targets,
            `"sparse"`: integer targets,
            `None`: no targets get yielded (only input images are yielded).
        batch_size: Integer, size of a batch.
        shuffle: Boolean, whether to shuffle the data between epochs.
        seed: Random seed for data shuffling.
        data_format: String, one of `channels_first`, `channels_last`.
        save_to_dir: Optional directory where to save the pictures
            being yielded, in a viewable format. This is useful
            for visualizing the random transformations being
            applied, for debugging purposes.
        save_prefix: String prefix to use for saving sample
            images (if `save_to_dir` is set).
        save_format: Format to use for saving sample images
            (if `save_to_dir` is set).
    """

    def __init__(self, image_lists, image_data_generator,
                 category, image_dir,
                 target_size=(256, 256), color_mode='rgb',
                 class_mode='categorical',
                 batch_size=32, shuffle=True, seed=None,
                 data_format=None,
                 save_to_dir=None, save_prefix='', save_format='jpeg'):
        if data_format is None:
            data_format = K.image_data_format()

        classes = list(image_lists.keys())
        self.category = category
        self.num_classes = len(classes)
        self.image_lists = image_lists
        self.image_dir = image_dir

        how_many_files = 0
        for label_name in classes:
            for _ in self.image_lists[label_name][category]:
                how_many_files += 1

        self.samples = how_many_files
        self.class_indices = dict(zip(classes, range(len(classes))))
        self.id2class = dict((v, k) for k, v in self.class_indices.items())
        self.classes = np.zeros((self.samples,), dtype='int32')

        self.image_data_generator = image_data_generator
        self.target_size = tuple(target_size)
        if color_mode not in {'rgb', 'grayscale'}:
            raise ValueError('Invalid color mode:', color_mode,
                             '; expected "rgb" or "grayscale".')
        self.color_mode = color_mode
        self.data_format = data_format
        if self.color_mode == 'rgb':
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (3,)
            else:
                self.image_shape = (3,) + self.target_size
        else:
            if self.data_format == 'channels_last':
                self.image_shape = self.target_size + (1,)
            else:
                self.image_shape = (1,) + self.target_size

        if class_mode not in {'categorical', 'binary', 'sparse', None}:
            raise ValueError('Invalid class_mode:', class_mode,
                             '; expected one of "categorical", '
                             '"binary", "sparse", or None.')
        self.class_mode = class_mode
        self.save_to_dir = save_to_dir
        self.save_prefix = save_prefix
        self.save_format = save_format

        i = 0
        self.filenames = []
        for label_name in classes:
            for j, _ in enumerate(self.image_lists[label_name][category]):
                self.classes[i] = self.class_indices[label_name]
                img_path = get_image_path(self.image_lists,
                                          label_name,
                                          j,
                                          self.image_dir,
                                          self.category)
                self.filenames.append(img_path)
                i += 1

        print("Found {} {} files".format(len(self.filenames), category))
        super(ImageListIterator, self).__init__(self.samples, batch_size, shuffle,
                                                seed)

    def next(self):
        """For python 2.x.

        # Returns
            The next batch.
        """
        # Keeps under lock only the mechanism which advances
        # the indexing of each batch.
        with self.lock:
            index_array = next(self.index_generator)
        # The transformation of images is not under thread lock
        # so it can be done in parallel
        return self._get_batches_of_transformed_samples(index_array)

    def _get_batches_of_transformed_samples(self, index_array):
        current_batch_size=len(index_array)
        batch_x = np.zeros((current_batch_size,) + self.image_shape,
                           dtype=K.floatx())
        grayscale = self.color_mode == 'grayscale'
        # build batch of image data
        for i, j in enumerate(index_array):
            img = load_img(self.filenames[j],
                           grayscale=grayscale,
                           target_size=self.target_size)
            x = img_to_array(img, data_format=self.data_format)
            x = self.image_data_generator.random_transform(x)
            x = self.image_data_generator.standardize(x)
            batch_x[i] = x
        # optionally save augmented images to disk for debugging purposes
        if self.save_to_dir:
            for i, j in enumerate(index_array):
                img = array_to_img(batch_x[i], self.data_format, scale=True)
                fname = '{prefix}_{index}_{hash}.{format}'.format(
                    prefix=self.save_prefix,
                    index=j,
                    hash=np.random.randint(10000),
                    format=self.save_format)
                img.save(os.path.join(self.save_to_dir, fname))
        # build batch of labels
        if self.class_mode == 'sparse':
            batch_y = self.classes[index_array]
        elif self.class_mode == 'binary':
            batch_y = self.classes[index_array].astype(K.floatx())
        elif self.class_mode == 'categorical':
            batch_y = np.zeros((len(batch_x), self.num_classes),
                               dtype=K.floatx())
            for i, label in enumerate(self.classes[index_array]):
                batch_y[i, label] = 1.
        else:
            return batch_x
        return batch_x, batch_y

# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/image_retraining/retrain.py
def get_image_path(image_lists, label_name, index, image_dir, category):
    """"Returns a path to an image for a label at the given index.

    # Arguments
      image_lists: Dictionary of training images for each label.
      label_name: Label string we want to get an image for.
      index: Int offset of the image we want. This will be moduloed by the
      available number of images for the label, so it can be arbitrarily large.
      image_dir: Root folder string of the subfolders containing the training
      images.
      category: Name string of set to pull images from - training, testing, or
      validation.

    # Returns
      File system path string to an image that meets the requested parameters.
    """
    if label_name not in image_lists:
        raise ValueError('Label does not exist ', label_name)
    label_lists = image_lists[label_name]
    if category not in label_lists:
        raise ValueError('Category does not exist ', category)
    category_list = label_lists[category]
    if not category_list:
        raise ValueError('Label %s has no images in the category %s.',
                         label_name, category)
    mod_index = index % len(category_list)
    base_name = category_list[mod_index]
    sub_dir = label_lists['dir']
    full_path = os.path.join(image_dir, sub_dir, base_name)
    return full_path

In [13]:
BATCH_SIZE=32
RANDOM_SEED=123

train_datagen = CustomImageDataGenerator(rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest',
  preprocessing_function=preprocessing_function)

test_datagen = CustomImageDataGenerator(preprocessing_function=preprocessing_function)

train_generator = train_datagen.flow_from_image_lists(
    image_lists=image_lists,
    category='training',
    image_dir=DATA_DIR,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',seed=RANDOM_SEED)

val_generator = test_datagen.flow_from_image_lists(
    image_lists=image_lists,
    category='validation',
    image_dir=DATA_DIR,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',seed=RANDOM_SEED)

Found 3030 training files
Found 5647 validation files


In [14]:
N_CLASS=val_generator.num_classes
nb_train_samples=train_generator.samples
nb_validation_samples=val_generator.samples
print(N_CLASS,nb_train_samples,nb_validation_samples)

class_to_idx=val_generator.class_indices
idx_to_class={class_to_idx[cls]:cls for cls in class_to_idx}
print(idx_to_class)

from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
               'balanced',
                np.unique(train_generator.classes), 
                train_generator.classes)

101 3030 5647
{0: 'accordion', 1: 'airplanes', 2: 'anchor', 3: 'ant', 4: 'barrel', 5: 'bass', 6: 'beaver', 7: 'binocular', 8: 'bonsai', 9: 'brain', 10: 'brontosaurus', 11: 'buddha', 12: 'butterfly', 13: 'camera', 14: 'cannon', 15: 'car side', 16: 'ceiling fan', 17: 'cellphone', 18: 'chair', 19: 'chandelier', 20: 'cougar body', 21: 'cougar face', 22: 'crab', 23: 'crayfish', 24: 'crocodile', 25: 'crocodile head', 26: 'cup', 27: 'dalmatian', 28: 'dollar bill', 29: 'dolphin', 30: 'dragonfly', 31: 'electric guitar', 32: 'elephant', 33: 'emu', 34: 'euphonium', 35: 'ewer', 36: 'faces', 37: 'faces easy', 38: 'ferry', 39: 'flamingo', 40: 'flamingo head', 41: 'garfield', 42: 'gerenuk', 43: 'gramophone', 44: 'grand piano', 45: 'hawksbill', 46: 'headphone', 47: 'hedgehog', 48: 'helicopter', 49: 'ibis', 50: 'inline skate', 51: 'joshua tree', 52: 'kangaroo', 53: 'ketch', 54: 'lamp', 55: 'laptop', 56: 'leopards', 57: 'llama', 58: 'lobster', 59: 'lotus', 60: 'mandolin', 61: 'mayfly', 62: 'menorah', 63

In [16]:
start_epoch=0
for l in base_model.layers:
    l.trainable=False
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

mc = ModelCheckpoint(net_description+'.h5', monitor='val_acc', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_acc',patience=2)
FIRST_EPOCHS=10
#tb, 
hist1=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=FIRST_EPOCHS, verbose=1, 
                    initial_epoch=0, callbacks=[mc, es], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 456, 456, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 228, 228, 48) 1296        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 228, 228, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 228, 228, 48) 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

Epoch 1/10
94/94 [==============================] - 418s 4s/step - loss: 3.6445 - acc: 0.3984 - val_loss: 1.6820 - val_acc: 0.8702

Epoch 00001: val_acc improved from -inf to 0.87021, saving model to enet5_train.h5
Epoch 2/10
94/94 [==============================] - 407s 4s/step - loss: 1.8612 - acc: 0.7981 - val_loss: 0.6670 - val_acc: 0.8928

Epoch 00002: val_acc improved from 0.87021 to 0.89279, saving model to enet5_train.h5
Epoch 3/10
94/94 [==============================] - 405s 4s/step - loss: 1.0973 - acc: 0.8667 - val_loss: 0.4140 - val_acc: 0.8987

Epoch 00003: val_acc improved from 0.89279 to 0.89866, saving model to enet5_train.h5
Epoch 4/10
94/94 [==============================] - 405s 4s/step - loss: 0.8436 - acc: 0.8807 - val_loss: 0.3308 - val_acc: 0.9013

Epoch 00004: val_acc improved from 0.89866 to 0.90134, saving model to enet5_train.h5
Epoch 5/10
94/94 [==============================] - 405s 4s/step - loss: 0.6842 - acc: 0.8920 - val_loss: 0.2893 - val_acc: 0.9058


In [11]:
#DOES NOT WORK!!!
start_epoch=len(hist1.history['loss'])
#start_epoch=2

model.load_weights(net_description+'.h5')
if True:
    for l in base_model.layers:
        l.trainable=True
else:
    trainable=False
    for layer in base_model.layers:
        if layer.name=='block7c_expand_conv':
            trainable=True
        layer.trainable=trainable

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
model.summary()

mc = ModelCheckpoint(net_description+'_ft.h5', monitor='val_acc', verbose=1, save_best_only=True)
es=EarlyStopping(monitor='val_acc',patience=2 )
SECOND_EPOCHS=5+start_epoch

hist2=model.fit_generator(train_generator, steps_per_epoch=nb_train_samples//BATCH_SIZE, epochs=SECOND_EPOCHS, verbose=1, 
                    initial_epoch=start_epoch, callbacks=[mc], validation_data=val_generator, validation_steps=nb_validation_samples // BATCH_SIZE,class_weight=class_weights)

NameError: name 'hist1' is not defined